### Dati festival Sanremo

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

data_list = []
year_start = 1951
year_end = 2024  # escluso

for year in range(year_start, year_end):
    url = f'https://it.wikipedia.org/wiki/Festival_di_Sanremo_{year}'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Cerca la prima tabella significativa
        table = soup.find('table', class_='infobox') or soup.find('table', class_='wikitable')
        
        if not table:
            print(year, "- Nessuna tabella informativa trovata.")
            continue

        # Analizza tutte le righe della tabella
        rows = table.find_all('tr')
        data = {}

        for row in rows:
            th = row.find('th')
            td = row.find('td')
            if th and td:
                label = th.get_text(strip=True)
                data[label] = td

        try:
            periodo = data.get('Periodo', None)
            sede = data.get('Sede', None)
            presentatore = data.get('Presentatore', None)
            vincitore = data.get('Vincitore', None)
            partecipanti = data.get('Partecipanti', None)

            # Estrai il testo pulito
            periodo_txt = periodo.get_text(strip=True) if periodo else 'Non trovato'
            sede_txt = sede.find('a')['title'] if sede and sede.find('a') else 'Non trovato'
            presentatore_txt = presentatore.find('a')['title'] if presentatore and presentatore.find('a') else 'Non trovato'
            vincitore_txt = vincitore.find('a')['title'] if vincitore and vincitore.find('a') else 'Non trovato'
            partecipanti_txt = partecipanti.get_text(strip=True) if partecipanti else 'Non trovato'

            data_list.append({
                'Anno': year,
                'Periodo': periodo_txt,
                'Sede': sede_txt,
                'Presentatore': presentatore_txt,
                'Partecipanti': partecipanti_txt,
                'Vincitore': vincitore_txt
            })

        except Exception as e:
            print(year, "- Errore durante l'estrazione dinamica:", e)
    else:
        print(year, "- Pagina non disponibile.")

# Crea DataFrame
df_sanremo = pd.DataFrame(data_list)

print("Estrazione completata.")


Estrazione completata.


In [71]:
#data_list

In [17]:
df_sanremo

,Anno,Periodo,Sede,Presentatore,Partecipanti,Vincitore
0,1951,29-31 gennaio,Casinò di Sanremo,Nunzio Filogamo,3,Nilla Pizzi
1,1952,28-30 gennaio,Casinò di Sanremo,Nunzio Filogamo,5,Nilla Pizzi
2,1953,29-31 gennaio,Casinò di Sanremo,Nunzio Filogamo,10,Carla Boni
3,1954,28-30 gennaio,Casinò di Sanremo,Nunzio Filogamo,12,Gino Latilla
4,1955,27-29 gennaio,Casinò di Sanremo,Armando Pizzo,15,Claudio Villa
...,...,...,...,...,...,...
68,2019,5 - 9 febbraio,Teatro Ariston,Claudio Baglioni,24,Mahmood
69,2020,4 - 8 febbraio,Teatro Ariston,Amadeus (conduttore televisivo),32,Diodato (cantante)
70,2021,2 - 6 marzo,Teatro Ariston,Amadeus (conduttore televisivo),34,Måneskin
71,2022,1º - 5 febbraio,Teatro Ariston,Amadeus (conduttore televisivo),25,Mahmood


In [4]:
df_sanremo['Anno'] = df_sanremo['Anno'].astype(int)

In [99]:
# df_sanremo.head(5)

In [18]:
print(df_sanremo['Periodo'].head(10))

0               29-31 gennaio
1               28-30 gennaio
2               29-31 gennaio
3               28-30 gennaio
4               27-29 gennaio
5                  8-10 marzo
6                7-9 febbraio
7    30 gennaio - 1º febbraio
8               29-31 gennaio
9               28-30 gennaio
Name: Periodo, dtype: object


Formato misto della colonna 'Periodo'
Osserviamo che la colonna 'Periodo' presenta un formato eterogeneo.
Alcune righe indicano un intervallo di date (es. '29-31 gennaio'),
mentre altre sembrano indicare un periodo più specifico o potenzialmente
una singola data o un intervallo con una formattazione diversa (es. '8-10 marzo', '7-9 febbraio').
Questa inconsistenza nel formato richiederà una gestione specifica se vogliamo
estrarre informazioni come la data di inizio e la data di fine in modo uniforme.

In [20]:
df_sanremo.loc[df_sanremo['Anno'] == 1968, 'Periodo'] = '31 gennaio - 2 febbraio'

In [21]:
# Sostituisce tutti i tipi di trattini con quello standard '-'
df_sanremo['Periodo'] = df_sanremo['Periodo'].str.replace(r'[–—‒−]', '-', regex=True)

In [22]:
# Divide la colonna "Periodo" in due colonne
df_sanremo[['giorno_inizio', 'giorno_fine']] = df_sanremo['Periodo'].str.split('-', expand=True)

# Pulisce caratteri speciali e spazi
df_sanremo['giorno_inizio'] = df_sanremo['giorno_inizio'].str.replace('º', '', regex=False).str.strip()
df_sanremo['giorno_fine'] = df_sanremo['giorno_fine'].str.replace('º', '', regex=False).str.strip()

# Estrai giorno e mese per inizio
df_sanremo[['giorno_inizio_num', 'mese_inizio']] = df_sanremo['giorno_inizio'].str.split(' ', expand=True)

# Estrai giorno e (eventualmente) mese per fine
giorno_fine_split = df_sanremo['giorno_fine'].str.split(' ', expand=True)
df_sanremo['giorno_fine_num'] = giorno_fine_split[0]
df_sanremo['mese_fine'] = giorno_fine_split[1]

# Se mese_fine è NaN, copia il mese da inizio
df_sanremo['mese_fine'] = df_sanremo['mese_fine'].fillna(df_sanremo['mese_inizio'])
df_sanremo['mese_inizio'] = df_sanremo['mese_inizio'].fillna(df_sanremo['mese_fine'])

# Crea le nuove colonne con date in formato stringa
df_sanremo['data_inizio'] = df_sanremo['giorno_inizio_num'] + ' ' + df_sanremo['mese_inizio'] + ' ' + df_sanremo['Anno'].astype(str)
df_sanremo['data_fine'] = df_sanremo['giorno_fine_num'] + ' ' + df_sanremo['mese_fine']+ ' ' + df_sanremo['Anno'].astype(str)


In [23]:
display(df_sanremo)

,Anno,Periodo,Sede,Presentatore,Partecipanti,Vincitore,giorno_inizio,giorno_fine,giorno_inizio_num,mese_inizio,giorno_fine_num,mese_fine,data_inizio,data_fine
0,1951,29-31 gennaio,Casinò di Sanremo,Nunzio Filogamo,3,Nilla Pizzi,29,31 gennaio,29,gennaio,31,gennaio,29 gennaio 1951,31 gennaio 1951
1,1952,28-30 gennaio,Casinò di Sanremo,Nunzio Filogamo,5,Nilla Pizzi,28,30 gennaio,28,gennaio,30,gennaio,28 gennaio 1952,30 gennaio 1952
2,1953,29-31 gennaio,Casinò di Sanremo,Nunzio Filogamo,10,Carla Boni,29,31 gennaio,29,gennaio,31,gennaio,29 gennaio 1953,31 gennaio 1953
3,1954,28-30 gennaio,Casinò di Sanremo,Nunzio Filogamo,12,Gino Latilla,28,30 gennaio,28,gennaio,30,gennaio,28 gennaio 1954,30 gennaio 1954
4,1955,27-29 gennaio,Casinò di Sanremo,Armando Pizzo,15,Claudio Villa,27,29 gennaio,27,gennaio,29,gennaio,27 gennaio 1955,29 gennaio 1955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,2019,5 - 9 febbraio,Teatro Ariston,Claudio Baglioni,24,Mahmood,5,9 febbraio,5,febbraio,9,febbraio,5 febbraio 2019,9 febbraio 2019
69,2020,4 - 8 febbraio,Teatro Ariston,Amadeus (conduttore televisivo),32,Diodato (cantante),4,8 febbraio,4,febbraio,8,febbraio,4 febbraio 2020,8 febbraio 2020
70,2021,2 - 6 marzo,Teatro Ariston,Amadeus (conduttore televisivo),34,Måneskin,2,6 marzo,2,marzo,6,marzo,2 marzo 2021,6 marzo 2021
71,2022,1º - 5 febbraio,Teatro Ariston,Amadeus (conduttore televisivo),25,Mahmood,1,5 febbraio,1,febbraio,5,febbraio,1 febbraio 2022,5 febbraio 2022


In [24]:
missing_values = df_sanremo['giorno_fine'].isnull().sum()
print("Numero di valori mancanti nella colonna giorno_fine:", missing_values)

Numero di valori mancanti nella colonna giorno_fine: 0


In [25]:
df_sanremo = df_sanremo.drop(columns=['giorno_inizio', 'giorno_fine', 'giorno_inizio_num', 'mese_inizio', 'giorno_fine_num', 'mese_fine'])

In [26]:
df_sanremo

,Anno,Periodo,Sede,Presentatore,Partecipanti,Vincitore,data_inizio,data_fine
0,1951,29-31 gennaio,Casinò di Sanremo,Nunzio Filogamo,3,Nilla Pizzi,29 gennaio 1951,31 gennaio 1951
1,1952,28-30 gennaio,Casinò di Sanremo,Nunzio Filogamo,5,Nilla Pizzi,28 gennaio 1952,30 gennaio 1952
2,1953,29-31 gennaio,Casinò di Sanremo,Nunzio Filogamo,10,Carla Boni,29 gennaio 1953,31 gennaio 1953
3,1954,28-30 gennaio,Casinò di Sanremo,Nunzio Filogamo,12,Gino Latilla,28 gennaio 1954,30 gennaio 1954
4,1955,27-29 gennaio,Casinò di Sanremo,Armando Pizzo,15,Claudio Villa,27 gennaio 1955,29 gennaio 1955
...,...,...,...,...,...,...,...,...
68,2019,5 - 9 febbraio,Teatro Ariston,Claudio Baglioni,24,Mahmood,5 febbraio 2019,9 febbraio 2019
69,2020,4 - 8 febbraio,Teatro Ariston,Amadeus (conduttore televisivo),32,Diodato (cantante),4 febbraio 2020,8 febbraio 2020
70,2021,2 - 6 marzo,Teatro Ariston,Amadeus (conduttore televisivo),34,Måneskin,2 marzo 2021,6 marzo 2021
71,2022,1º - 5 febbraio,Teatro Ariston,Amadeus (conduttore televisivo),25,Mahmood,1 febbraio 2022,5 febbraio 2022


#pip install genderize

In [15]:
"""from genderize import Genderize

# Funzione per estrarre il primo nome "pulito"
def estrai_primo_nome(nome_completo):
    if pd.isnull(nome_completo):
        return None
    nome_pulito = nome_completo.split('(')[0].strip()
    return nome_pulito.split()[0]

# Estrai primo nome da Presentatore e Vincitore
df_sanremo['Nome_Presentatore'] = df_sanremo['Presentatore'].apply(estrai_primo_nome)
df_sanremo['Nome_Vincitore'] = df_sanremo['Vincitore'].apply(estrai_primo_nome)

# Unisci tutti i nomi unici da analizzare
nomi_unici = pd.Series(df_sanremo['Nome_Presentatore'].tolist() + df_sanremo['Nome_Vincitore'].tolist()).dropna().unique().tolist()

# Chiamata a genderize.io
gender_data = Genderize().get(nomi_unici)

# Costruisci dizionario {nome: genere}
gender_dict = {item['name']: item['gender'] for item in gender_data if item['gender'] is not None}

# Aggiungi eccezioni manuali se necessario
gender_dict['Amadeus'] = 'M'
gender_dict['Måneskin'] = None  # gruppo musicale
gender_dict['Matia'] = 'F'  # per Matia Bazar

# Mappa i generi nel DataFrame
df_sanremo['Genere_Presentatore'] = df_sanremo['Nome_Presentatore'].map(gender_dict)
df_sanremo['Genere_Vincitore'] = df_sanremo['Nome_Vincitore'].map(gender_dict)

# Visualizza i primi risultati
print(df_sanremo[['Anno', 'Presentatore', 'Genere_Presentatore', 'Vincitore', 'Genere_Vincitore', 'data_inizio', 'data_fine']].head(10))
"""

'from genderize import Genderize\n\n# Funzione per estrarre il primo nome "pulito"\ndef estrai_primo_nome(nome_completo):\n    if pd.isnull(nome_completo):\n        return None\n    nome_pulito = nome_completo.split(\'(\')[0].strip()\n    return nome_pulito.split()[0]\n\n# Estrai primo nome da Presentatore e Vincitore\ndf_sanremo[\'Nome_Presentatore\'] = df_sanremo[\'Presentatore\'].apply(estrai_primo_nome)\ndf_sanremo[\'Nome_Vincitore\'] = df_sanremo[\'Vincitore\'].apply(estrai_primo_nome)\n\n# Unisci tutti i nomi unici da analizzare\nnomi_unici = pd.Series(df_sanremo[\'Nome_Presentatore\'].tolist() + df_sanremo[\'Nome_Vincitore\'].tolist()).dropna().unique().tolist()\n\n# Chiamata a genderize.io\ngender_data = Genderize().get(nomi_unici)\n\n# Costruisci dizionario {nome: genere}\ngender_dict = {item[\'name\']: item[\'gender\'] for item in gender_data if item[\'gender\'] is not None}\n\n# Aggiungi eccezioni manuali se necessario\ngender_dict[\'Amadeus\'] = \'M\'\ngender_dict[\'Måneski

In [66]:
df_sanremo.to_excel('dati-festival-sanremo-1951-2023.xlsx',index=False)